# Algoritmo de BackPropagation multiclase

Este codigo se utilizará para entrenar la red neuronal de clasificacion de Nuestras Caras



### Código en Python del Algoritmo de BackPropagation

La clase del dataset debe ser categorica
Existe una clase en Python que resuelve el problema

In [ ]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/.drive')
!mkdir -p "/content/.drive/My Drive/DMA"
!mkdir -p "/content/buckets"
!ln -s "/content/.drive/My Drive/DMA" /content/buckets/b1

In [ ]:
# instalo  itables solo si no esta instalado
!pip show itables >/dev/null || pip install itables

In [ ]:
import polars as pl
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time
import os
import pickle
from functools import reduce
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)


In [ ]:
# definicion de la clase de graficos

class perceptron_plot:
    """plotting first hidden layer class"""
    def __init__(self, X, Y, delay) -> None:
        self.X = X
        self.Y = Y
        self.delay = delay
        x1_min = np.min(X[:,0])
        x2_min = np.min(X[:,1])
        x1_max = np.max(X[:,0])
        x2_max = np.max(X[:,1])
        self.x1_min = x1_min - 0.1*(x1_max - x1_min)
        self.x1_max = x1_max + 0.1*(x1_max - x1_min)
        self.x2_min = x2_min - 0.1*(x2_max - x2_min)
        self.x2_max = x2_max + 0.1*(x2_max - x2_min)
        self.fig = plt.figure(figsize = (10,8))
        self.ax = self.fig.subplots()
        self.ax.set_xlim(self.x1_min, self.x1_max, auto=False)
        self.ax.set_ylim(self.x2_min, self.x2_max, auto=False)

    def graficarVarias(self, W, x0, epoch, error) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + "{0:.2E}".format(error))
        # ploteo puntos
        num_classes = len(np.unique(self.Y))
        # mycolors = plt.cm.get_cmap('tab10', num_classes)
        unique_labels = np.unique(self.Y)
        label_mapping = {label: i for i, label in enumerate(unique_labels)}
        numerical_labels = np.array([label_mapping[label] for label in self.Y])
        # scatter = self.ax.scatter(self.X[:,0], self.X[:,1], c=self.Y, s=20)
        scatter = self.ax.scatter(self.X[:,0], self.X[:,1], c=numerical_labels, s=20, cmap=plt.get_cmap('viridis', num_classes))
        # self.ax.plot(self.X[:,0], self.X[:,1], 'o', c=vcolores,  markersize=2)


        # dibujo las rectas
        for i in range(len(x0)):
            #vx2_min = -(W[0,i]*self.x1_min + x0[i])/W[1,i]
            #vx2_max = -(W[0,i]*self.x1_max + x0[i])/W[1,i]
            vx2_min = -(W[i,0]*self.x1_min + x0[i])/W[i,1]
            vx2_max = -(W[i,0]*self.x1_max + x0[i])/W[i,1]

            self.ax.plot([self.x1_min, self.x1_max],
                         [vx2_min, vx2_max],
                         linewidth = 2,
                         color = 'red',
                         alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


In [ ]:
# definicion de las funciones de activacion
#  y sus derivadas
#  ahora agregando las versiones VECTORIZADAS

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + math.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
    return y

# version vectorizada de func_eval
func_eval_vec = np.vectorize(func_eval)


def deriv_eval(fname, y):  #atencion que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
    return d


# version vectorizada de deriv_eval
deriv_eval_vec = np.vectorize(deriv_eval)

### Clase  multiperceptron
entrenar, predecir

In [ ]:
# definicion de la clase de multiperceptron

class multiperceptron(object):
    """Multiperceptron class"""

    # inicializacion de los pesos de todas las capas
    def _red_init(self, semilla) -> None:
        niveles = self.red['arq']['layers_qty']

        np.random.seed(semilla)
        for i in range(niveles):
           nivel = dict()
           nivel['id'] = i
           nivel['last'] = (i==(niveles-1))
           nivel['size'] = self.red["arq"]["layers_size"][i]
           nivel['func'] = self.red["arq"]["layers_func"][i]

           if( i==0 ):
              entrada_size = self.red['arq']['input_size']
           else:
              entrada_size =  self.red['arq']['layers_size'][i-1]

           salida_size =  nivel['size']

           # los pesos, inicializados random
           nivel['W'] = np.random.uniform(-0.5, 0.5, [salida_size, entrada_size])
           nivel['w0'] = np.random.uniform(-0.5, 0.5, [salida_size, 1])

           # los momentos, inicializados en CERO
           nivel['W_m'] = np.zeros([salida_size, entrada_size])
           nivel['w0_m'] = np.zeros([salida_size, 1])

           self.red['layer'].append(nivel)

    # constructor generico
    def __init__(self) -> None:
        self.data = dict()
        self.red = dict()
        self.carpeta = ""


    # inicializacion full
    def inicializar(self, df, campos, clase, hidden_layers_sizes, layers_func,
                 semilla, carpeta) -> None:

        # genero self.data
        self.data['X'] = np.array( df.select(campos))
        X_mean = self.data['X'].mean(axis=0)
        X_sd = self.data['X'].std(axis=0)
        self.data['X'] = (self.data['X'] - X_mean)/X_sd

        #  Ylabel en  numpy
        label =df.select(clase)
        self.data['Ylabel'] = np.array(label).reshape(len(label))

        # one-hot-encoding de Y
        col_originales = df.columns
        self.data['Y'] = np.array( df.to_dummies(clase).drop(col_originales, strict=False) )

        col_dummies = sorted( list( set(df.to_dummies(clase).columns) -  set(col_originales)))
        clases_originales = reduce(lambda acc, x: acc + [x[(len(clase)+1):]], col_dummies, [])

        tamanos = hidden_layers_sizes
        tamanos.append(self.data['Y'].shape[1])

        arquitectura = {
             'input_size' : self.data['X'].shape[1],
             'input_mean' : X_mean,
             'input_sd' :  X_sd,
             'output_values' : clases_originales,
             'layers_qty' : len(hidden_layers_sizes), # incluye la capa de salida, pero no la de entrada
             'layers_size' : tamanos ,
             'layers_func' : layers_func,
        }

        self.red['arq'] = arquitectura


        # inicializo  work
        self.red['work'] = dict()
        self.red['work']['epoch'] = 0
        self.red['work']['MSE'] = float('inf')
        self.red['work']['train_error_rate'] = float('inf')

        self.red['layer'] = list()
        self._red_init(semilla)

        # grabo el entorno
        self.carpeta = carpeta
        os.makedirs(self.carpeta, exist_ok=True)
        with open(self.carpeta+"/data.pkl", 'wb') as f:
            pickle.dump(self.data, f)

        with open(self.carpeta+"/red.pkl", 'wb') as f:
            pickle.dump(self.red, f)





    # Algoritmo Backpropagation
    def  entrenar(self, epoch_limit, MSE_umbral,
               learning_rate, lr_momento, save_frequency,
               retomar=True) -> None:

        # si debo retomar
        if( retomar):
            with open(self.carpeta+"/data.pkl", 'rb') as f:
              self.data = pickle.load(f)

            with open(self.carpeta+"/red.pkl", 'rb') as f:
              self.red = pickle.load(f)


        # inicializaciones del bucle principal del backpropagation
        epoch = self.red['work']['epoch']
        MSE = self.red['work']['MSE']

        # inicializacion del grafico
        grafico = perceptron_plot(X=self.data['X'], Y=self.data['Ylabel'], delay=0.1)

        # continuo mientras error cuadratico medio muy grande  y NO llegué al límite de epochs
        Xfilas = self.data['X'].shape[0]
        niveles = self.red["arq"]["layers_qty"]

        while ( MSE > MSE_umbral) and (epoch < epoch_limit) :
          epoch += 1


          # recorro siempre TODOS los registros de entrada
          for fila in range(Xfilas):
             # fila es el registro actual
             x = self.data['X'][fila:fila+1,:]
             clase = self.data['Y'][fila:fila+1,:]

             # propagar el x hacia adelante, FORWARD
             entrada = x.T  # la entrada a la red

             # etapa forward
             # recorro hacia adelante, nivel a nivel
             vsalida =  [0] *(niveles) # salida de cada nivel de la red

             for i in range(niveles):
               estimulos = self.red['layer'][i]['W'] @ entrada + self.red['layer'][i]['w0']
               vsalida[i] =  func_eval_vec(self.red['layer'][i]['func'], estimulos)
               entrada = vsalida[i]  # para la proxima vuelta


             # etapa backward
             # calculo los errores en la capa hidden y la capa output
             verror =  [0] *(niveles+1) # inicializo dummy
             verror[niveles] = clase.T - vsalida[niveles-1]

             i = niveles-1
             verror[i] = verror[i+1] * deriv_eval_vec(self.red['layer'][i]['func'], vsalida[i])

             for i in reversed(range(niveles-1)):
               verror[i] = deriv_eval_vec(self.red['layer'][i]['func'], vsalida[i])*(self.red['layer'][i+1]['W'].T @ verror[i+1])

             # ya tengo los errores que comete cada capa
             # corregir matrices de pesos, voy hacia atras
             # backpropagation
             entrada = x.T
             for i in range(niveles):
               self.red['layer'][i]['W_m'] = learning_rate *(verror[i] @ entrada.T) + lr_momento *self.red['layer'][i]['W_m']
               self.red['layer'][i]['w0_m'] = learning_rate * verror[i] + lr_momento * self.red['layer'][i]['w0_m']

               self.red['layer'][i]['W']  =  self.red['layer'][i]['W'] + self.red['layer'][i]['W_m']
               self.red['layer'][i]['w0'] =  self.red['layer'][i]['w0'] + self.red['layer'][i]['w0_m']
               entrada = vsalida[i]  # para la proxima vuelta



          # ya recalcule las matrices de pesos
          # ahora avanzo la red, feed-forward
          # para calcular el red(X) = Y
          entrada = self.data['X'].T
          for i in range(niveles):
            estimulos = self.red['layer'][i]['W'] @ entrada + self.red['layer'][i]['w0']
            salida =  func_eval_vec(self.red['layer'][i]['func'], estimulos)
            entrada = salida  # para la proxima vuelta

          # calculo el error cuadratico medio TODOS los X del dataset
          MSE= np.mean( (self.data['Y'].T - salida)**2 )

          # Grafico las rectas SOLAMENTE de la Primera Hidden Layer
          # tengo que hacer w0.T[0]  para que pase el vector limpio
          if( epoch % save_frequency == 0 ) or ( MSE <= MSE_umbral) or (epoch >= epoch_limit) :
              # grafico
              W = self.red['layer'][0]['W']
              w0 = self.red['layer'][0]['w0']
              grafico.graficarVarias(W, w0.T[0], epoch, MSE)

              # almaceno en work
              self.red['work']['epoch'] = epoch
              self.red['work']['MSE'] = MSE
              prediccion = np.argmax( salida.T, axis=1)
              # prediccion
              out = np.array(self.red["arq"]['output_values'])
              error_rate = np.mean( self.data['Ylabel'] != out[prediccion])
              self.red["work"]['train_error_rate'] = error_rate # error_rate != error cuadratico medio

              # grabo a un archivo la red neuronal  entrenada por donde esté
              #   solo la red, NO los datos
              with open(carpeta+"/red.pkl", 'wb') as f:
                 pickle.dump(self.red, f)

        return (epoch, MSE, self.red['work']['train_error_rate'] )


    # predigo a partir de modelo recien entrenado
    def  predecir(self, df_new, campos, clase) -> None:
        niveles = self.red['arq']['layers_qty']

        # etapa forward
        # recorro hacia adelante, nivel a nivel
        X_new =  np.array( df_new.select(campos))


        # estandarizo manualmente
        #  con las medias y desvios que almacene durante el entrenamiento
        X_new = (X_new - self.red['arq']['input_mean'])/self.red['arq']['input_sd']

        # grafico los datos nuevos
        Ylabel_new =df_new.select(clase)
        Ylabel_new = np.array(Ylabel_new).reshape(len(Ylabel_new))
        grafico = perceptron_plot(X=X_new, Y=Ylabel_new, delay=0.1)
        W = self.red['layer'][0]['W']
        w0 = self.red['layer'][0]['w0']
        grafico.graficarVarias(W, w0.T[0], epoch, MSE)

        # la entrada a la red,  el X que es TODO  x_new
        entrada = X_new.T  # traspongo, necesito vectores columna

        for i in range(niveles):
          estimulos = self.red['layer'][i]['W'] @ entrada + self.red['layer'][i]['w0']
          salida =  func_eval_vec(self.red['layer'][i]['func'], estimulos)
          entrada = salida  # para la proxima vuelta

        # me quedo con la neurona de la ultima capa que se activio con mayor intensidad
        pred_idx = np.argmax( salida.T, axis=1)
        pred_raw = np.max( salida.T, axis=1)

        # calculo error_rate
        out = np.array(self.red['arq']['output_values'])
        error_rate = np.mean( np.array(df_new.select("y") != out[pred_idx]))

        return (out[pred_idx], pred_raw, error_rate)


    # cargo un modelo ya entrenado, grabado en carpeta
    def cargar_modelo(self, carpeta) -> None:
        self.carpeta = carpeta

        with open(self.carpeta+"/red.pkl", 'rb') as f:
          self.red = pickle.load(f)

        return (self.red['work']['epoch'],
                self.red['work']['MSE'],
                self.red['work']['train_error_rate'] )


## 1 Lectura del Dataset

En este humilde y restringida version, la clase del dataset debe ser categorica, no es capaz de trabajar con clases continuas.
La clase categorica puede ser  n-aria

In [ ]:
!wget 'https://raw.githubusercontent.com/josekeh/dma_g2/main/input/caras.pkl'
!wget 'https://raw.githubusercontent.com/josekeh/dma_g2/main/input/caras50.pkl'

In [ ]:
import pickle

with open("caras50-megalimpio.pkl", "rb") as file:
    caras = pickle.load(file)

In [ ]:
# Lectura del dataset con la moderna libreria Polars  (Pandas debe extinguirse!)

# df = pl.read_csv('https://raw.githubusercontent.com/josekeh/dma_g2/main/input/isomap_df.csv')
# df

### 1.1 Particion  training/testing



*   Es valido cambiar la *semilla_particion* para probar distintos <test, train> y asi estimar con mas precisión el error rate en testing  (Montecarlo Estimation)



In [ ]:
# CODIGO PROFE
# # particion del dataset en training/testing

# # semilla_particion = 102191
# semilla_particion = 19961002
# pct_train = 0.75  # ratio de registros que va a training


# def train_test_split_df(df, seed=0, test_size=0.2):
#     return df.with_columns(
#         pl.int_range(pl.len(), dtype=pl.UInt32)
#         .shuffle(seed=seed)
#         .gt(pl.len() * test_size)
#         .alias("split")
#     ).partition_by("split", include_key=False)


# (df_train, df_test) = train_test_split_df(df,
#                                           seed=semilla_particion,
#                                           test_size=pct_train)

# # imprimo los tamaños
# print("Train:", df_train.shape)
# print("Test:", df_test.shape)

In [ ]:
import random
from collections import Counter

# Algoritmo para chequear la distribución de nombres
def check_separar(training_nombres):
  # Contar la frecuencia de cada palabra
  contador = Counter(training_nombres)
  total_palabras = sum(contador.values())
  cantidad_diferentes = len(contador)  # Número de palabras únicas

  # Convertir a porcentaje
  frecuencias = {palabra: (conteo / total_palabras) * 100 for palabra, conteo in contador.items()}

  umbral = 100 / cantidad_diferentes

  # Mostrar el gráfico si no es muy dispar
  if (min(frecuencias.values()) < umbral*.75 or max(frecuencias.values()) > umbral*1.25):
    return 0
  else:
    # Graficar
    plt.figure(figsize=(12, 6))
    plt.bar(frecuencias.keys(), frecuencias.values(), color='skyblue')

    # Agregar línea horizontal en 100% / cantidad de palabras diferentes
    plt.axhline(y=umbral, color='red', linestyle='--', label=f"Ideal: {umbral:.2f}%")

    # Etiquetas
    plt.xlabel("Persona")
    plt.ylabel("Frecuencia (%)")
    plt.title("Distribución del training set según las personas")
    plt.ylim(0, max(frecuencias.values()) + 5)  # Ajuste del eje Y
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

# Algoritmo para separar training y testing suponiendo 2 listas
def separar_aleatorio(lista_caras, lista_nombres, porcentaje_training):
    if (len(lista_caras) != len(lista_nombres)):
      print("Error las listas deben tener la misma longitud")
    else:
      chequeo = 0

      while (chequeo == 0):
        training_caras = []
        training_nombres = []
        testing_caras = []
        testing_nombres = []

        for i in range(len(lista_caras)):
          aleatorio = random.random()
          if aleatorio <= porcentaje_training:
            training_caras.append(lista_caras[i])
            training_nombres.append(lista_nombres[i])
          else:
            testing_caras.append(lista_caras[i])
            testing_nombres.append(lista_nombres[i])
        chequeo = check_separar(training_nombres)

      return training_caras, training_nombres, testing_caras, testing_nombres



### ISOMAP

In [ ]:
import numpy as np
from sklearn.manifold import Isomap
from sklearn.datasets import make_swiss_roll
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def procesar_isomap(train_caras, train_nombres, test_caras, test_nombres, mejor_n_components, mejor_n_neighbors):
  data_training = []
  for j in range(len(train_caras)):
    data_training.append(train_caras[j])

  data_testing = []
  for j in range(len(test_caras)):
    data_testing.append(test_caras[j])

  # Convertimos las listas en matrices de tamaño (n, 900)
  data_training = np.vstack(data_training)
  data_testing = np.vstack(data_testing)

  isomap = Isomap(n_components=mejor_n_components, n_neighbors=mejor_n_neighbors)
  output_isomap_training = isomap.fit_transform(data_training)

  output_isomap_testing = isomap.transform(data_testing)

  df_train = pl.DataFrame(output_isomap_training)
  df_train.columns = ['x' + str(i + 1) for i in range(df_train.shape[1])]
  df_train = df_train.with_columns(pl.Series(train_nombres).alias("y"))

  df_test = pl.DataFrame(output_isomap_testing)
  df_test.columns = ['x' + str(i + 1) for i in range(df_test.shape[1])]
  df_test = df_test.with_columns(pl.Series(test_nombres).alias("y"))

  return df_train, df_test




### Construcción de

## 2  Entrenamiento del modelo

### 2.1  Inicializacion de la neural network



*   Es valido cambiar la *semilla_red* para arrancar el entrenamiento con distintas rectas iniciales


In [ ]:
# defino la red multiperceptron
carpeta = "/content/buckets/b1/nn01/"  # cambiar con cada corrida
semilla_red = 200177  # define las rectas iniciales

# una sola capa oculta de 2 neuronas  [2]
# la capa oculta y la final tienen ambas logsig de activacion
mp = multiperceptron()
mp.inicializar(
    df=df_train, campos=campos, clase="y",  # especificaion del dataset
    hidden_layers_sizes=[20 ,13],  # no va la capa final, solo hidden layers
    layers_func=['logsig','logsig','logsig'], # funciones de activacion de cada capa
    semilla=semilla_red,
    carpeta = carpeta
    )

### 2.2 Entrenamiento de la neural network = backpropagation (con gridsearch)


In [ ]:
def campos_func(n):
  campos = []
  for i in range(n):
    campos.append(f"x{i+1}")
  return campos

In [ ]:
# Corridas
file_path = '/content/buckets/b1/DMA/NuestrasCaras/corridas.csv'
try:
  df = pl.read_csv(file_path)
  list_of_iters = df.drop("epoch", "error_rate").rows()
except Exception as e:
  print(f"Error reading the CSV file: {e}")

In [ ]:
import datetime

inicio = datetime.datetime.now()

# seeds = [200177] # seeds a utilizar
# learning_rates = [0.2] #learning rates a probar
# epochs = [200,400] #Límites de epochs a usar
# umbrales = [0.00001] # umbrales de mse a probar
# capas = [1] # capas a usar, sin contar la última oculta, o sea, la red va a tener capa[i] + 1 (verificamos que usar una más overfitea mucho)
# neuronas = [20] # Neuronas a usar en las capas declaradas anteriormente, por ahora dejamos fija la última oculta con 13 (a revisar)
# isomap_comp = [35] # n_components de isomap a probar, por ahora mantenemos el número de vecinos fijos
# contador = 1 #Contador solamente para guardar los diferentes pkl de las redes
# check_iter_list = 'list_of_iters' in locals()
# func_out = ['logsig', 'tansig']
seeds = [200177] # seeds a utilizar
learning_rates = [0.2] #learning rates a probar
epochs = [100, 500, 700, 1000, 1500] #Límites de epochs a usar, PONER EN ORDEN CRECIENTE
umbrales = [0.0001] # umbrales de mse a probar
capas = [1] # capas a usar, sin contar la última oculta, o sea, la red va a tener capa[i] + 1 (verificamos que usar una más overfitea mucho)
neuronas = [20] # Neuronas a usar en las capas declaradas anteriormente, por ahora dejamos fija la última oculta con 13 (a revisar)
isomap_comp = [80] # n_components de isomap a probar, por ahora mantenemos el número de vecinos fijos
contador = 0 #Contador solamente para guardar los diferentes pkl de las redes
check_iter_list = 'list_of_iters' in locals()
func_out = ['logsig']

# Creamos las listas de nombres y caras para usar en la red
faces = [sublista[0] for sublista in caras]
names = [sublista[1] for sublista in caras]

# Creación del df para guardar los resultados que vamos obteniendo (y no morir en el intento)
# seeds <- epochs <- lr <- umbral <- neuronas <- capas (ToDo: revisar para reciclar epochs y particiones de seed, ahora está haciendo uno por cada iter)

data = {'seed': [], 'capa': [], 'neurona': [], 'umbral': [], 'lr': [], 'epoch': [], 'epoch_lim': [],  'componente': [], 'error_rate': [], 'func': []}
df_results = pl.DataFrame(data)

df_results = df_results.with_columns([
    pl.col("epoch").cast(pl.Int64),
    pl.col("epoch_lim").cast(pl.Int64),
    pl.col("seed").cast(pl.Int64),
    pl.col("capa").cast(pl.Int64),
    pl.col("neurona").cast(pl.Int64),
    pl.col("componente").cast(pl.Int64),
    pl.col("error_rate").cast(pl.Float64),
    pl.col("umbral").cast(pl.Float64),
    pl.col("lr").cast(pl.Float64),
    pl.col("func").cast(pl.Utf8)
])


for capa in capas:
  for neurona in neuronas:
    for umbral in umbrales:
      for lr in learning_rates:
        for mejor_n_components in isomap_comp:
          x_campos = campos_func(mejor_n_components)
          for epoch_lim in epochs:
            for f in func_out:
              for seed in seeds:
                if check_iter_list and (seed, capa, neurona, umbral, lr, epoch_lim, mejor_n_components) in list_of_iters:
                  print("Ya se ha corrido este ejemplo, pass")
                else:
                  contador += 1
                  # Train test
                  random.seed(seed)
                  train_caras, train_nombres, test_caras, test_nombres = separar_aleatorio(faces, names, 0.7)
                  df_train, df_test = procesar_isomap(train_caras, train_nombres, test_caras, test_nombres, mejor_n_components, mejor_n_neighbors = 30)

                  hidden_layers_sizes = [neurona] * capa
                  hidden_layers_sizes.append(13)
                  layers_func = ['logsig'] * (len(hidden_layers_sizes)) + [f]
                  print(layers_func)

                  # Entrenamiento

                  # defino la red multiperceptron
                  carpeta = "/content/buckets/b1/nn" + str(contador) + "/"  # cambiar con cada corrida
                  semilla_red = seed  # define las rectas iniciales

                  mp = multiperceptron()
                  mp.inicializar(
                      df=df_train, campos = x_campos, clase="y",  # especificaion del dataset
                      hidden_layers_sizes = hidden_layers_sizes,  # no va la capa final, solo hidden layers
                      layers_func=layers_func, # funciones de activacion de cada capa
                      semilla=semilla_red,
                      carpeta = carpeta
                      )

                  (epoch, MSE, train_error_rate) = mp.entrenar(
                    epoch_limit=epoch_lim,
                    MSE_umbral=umbral,
                    learning_rate=lr,
                    lr_momento=0.2,
                    save_frequency=100,
                    retomar=True)

                  # Predicción
                  (y_hat,y_raw, test_error_rate) = mp.predecir(df_new=df_test, campos=x_campos, clase='y')

                  new_row = pl.DataFrame({'seed': [seed], 'capa': [capa], 'neurona': [neurona], 'umbral': [umbral], 'lr': [lr], 'epoch': [epoch], 'epoch_lim': [epoch_lim], 'componente': [mejor_n_components], 'error_rate': [test_error_rate], 'func':[f] })
                  print(new_row)
                  df_results = pl.concat([df_results, new_row])

fin = datetime.datetime.now()
print(fin - inicio)



In [ ]:
df_results_in = df_results

In [ ]:
## Test reciclado de epochs

#inicio2 = datetime.datetime.now()
seeds = [200177, 1999, 1996, 1949] # seeds a utilizar
learning_rates = [0.2] #learning rates a probar
epochs = [100, 500, 700, 1000, 1500,2000] #Límites de epochs a usar, PONER EN ORDEN CRECIENTE
umbrales = [0.0001] # umbrales de mse a probar
capas = [1, 2] # capas a usar, sin contar la última oculta, o sea, la red va a tener capa[i] + 1 (verificamos que usar una más overfitea mucho)
neuronas = [20,25,30] # Neuronas a usar en las capas declaradas anteriormente, por ahora dejamos fija la última oculta con 13 (a revisar)
isomap_comp = [70, 80] # n_components de isomap a probar, por ahora mantenemos el número de vecinos fijos
contador = 0 #Contador solamente para guardar los diferentes pkl de las redes
check_iter_list = 'list_of_iters' in locals()
func_out = ['logsig', 'tansig']

#para test
# seeds = [200177] # seeds a utilizar
# learning_rates = [0.2] #learning rates a probar
# epochs = [100, 500, 700, 1000, 1500] #Límites de epochs a usar, PONER EN ORDEN CRECIENTE
# umbrales = [0.0001] # umbrales de mse a probar
# capas = [1] # capas a usar, sin contar la última oculta, o sea, la red va a tener capa[i] + 1 (verificamos que usar una más overfitea mucho)
# neuronas = [20] # Neuronas a usar en las capas declaradas anteriormente, por ahora dejamos fija la última oculta con 13 (a revisar)
# isomap_comp = [80] # n_components de isomap a probar, por ahora mantenemos el número de vecinos fijos
# contador = 0 #Contador solamente para guardar los diferentes pkl de las redes
# check_iter_list = 'list_of_iters' in locals()
# func_out = ['logsig']

# Creamos las listas de nombres y caras para usar en la red
faces = [sublista[0] for sublista in caras]
names = [sublista[1] for sublista in caras]

# Creación del df para guardar los resultados que vamos obteniendo (y no morir en el intento)
# seeds <- epochs <- lr <- umbral <- neuronas <- capas (ToDo: revisar para reciclar epochs y particiones de seed, ahora está haciendo uno por cada iter)

data = {'seed': [], 'capa': [], 'neurona': [], 'umbral': [], 'lr': [], 'epoch': [], 'epoch_lim': [],  'componente': [], 'error_rate': [], 'func': []}
df_results = pl.DataFrame(data)

df_results = df_results.with_columns([
    pl.col("epoch").cast(pl.Int64),
    pl.col("epoch_lim").cast(pl.Int64),
    pl.col("seed").cast(pl.Int64),
    pl.col("capa").cast(pl.Int64),
    pl.col("neurona").cast(pl.Int64),
    pl.col("componente").cast(pl.Int64),
    pl.col("error_rate").cast(pl.Float64),
    pl.col("umbral").cast(pl.Float64),
    pl.col("lr").cast(pl.Float64),
    pl.col("func").cast(pl.Utf8)
])


for capa in capas:
  for neurona in neuronas:
    for umbral in umbrales:
      for lr in learning_rates:
        for mejor_n_components in isomap_comp:
          x_campos = campos_func(mejor_n_components)
          for seed in seeds:
            for f in func_out:
              contador += 1
              epoch_init = True
              skip = False
              for epoch_lim in epochs:
                if check_iter_list and (seed, capa, neurona, umbral, lr, epoch_lim, mejor_n_components) in list_of_iters:
                  print("Ya se ha corrido este ejemplo, pass")
                else:
                  # Train test
                  random.seed(seed)
                  train_caras, train_nombres, test_caras, test_nombres = separar_aleatorio(faces, names, 0.7)
                  df_train, df_test = procesar_isomap(train_caras, train_nombres, test_caras, test_nombres, mejor_n_components, mejor_n_neighbors = 5)

                  hidden_layers_sizes = [neurona] * capa
                  hidden_layers_sizes.append(13)
                  layers_func = ['logsig'] * (len(hidden_layers_sizes)) + [f]
                  print(layers_func)

                  # Entrenamiento

                  # defino la red multiperceptron
                  carpeta = "/content/buckets/b1/nn" + str(contador)  # cambiar con cada corrida
                  semilla_red = seed  # define las rectas iniciales

                  if epoch_init:
                    epoch_init = False
                    mp = multiperceptron()
                    mp.inicializar(
                        df=df_train, campos = x_campos, clase="y",  # especificaion del dataset
                        hidden_layers_sizes = hidden_layers_sizes,  # no va la capa final, solo hidden layers
                        layers_func=layers_func, # funciones de activacion de cada capa
                        semilla=semilla_red,
                        carpeta = carpeta
                        )

                    (epoch, MSE, train_error_rate) = mp.entrenar(
                      epoch_limit=epoch_lim,
                      MSE_umbral=umbral,
                      learning_rate=lr,
                      lr_momento=0.2,
                      save_frequency=100,
                      retomar=True)

                    if epoch < epoch_lim:  #Si llega al limite antes no tiene sentido seguir corriendo epochs
                      skip = True
                  else: #reciclo el anterior
                    if not skip:
                      print('Hi')
                      new_lim = epoch_lim - epoch # el aumento en epochs
                      (epoch, MSE, train_error_rate) = mp.entrenar(
                        epoch_limit = epoch_lim, # aumento
                        MSE_umbral=umbral,
                        learning_rate=lr,
                        lr_momento=0.2,
                        save_frequency=100,
                        retomar=True)

                      if epoch < epoch_lim:
                        skip = True


                    else:
                      print('no')

                  # Predicción
                  (y_hat,y_raw, test_error_rate) = mp.predecir(df_new=df_test, campos=x_campos, clase='y')

                  new_row = pl.DataFrame({'seed': [seed], 'capa': [capa], 'neurona': [neurona], 'umbral': [umbral], 'lr': [lr], 'epoch': [epoch], 'epoch_lim': [epoch_lim], 'componente': [mejor_n_components], 'error_rate': [test_error_rate], 'func':[f] })
                  print(new_row)
                  df_results = pl.concat([df_results, new_row])
                  print(contador)


#fin2 = datetime.datetime.now()


In [ ]:
df_results

In [ ]:
# prompt: generate a code to summarize the df_result df. It has to do a mean of the error_rate grouping by capa1, capa2, umbral, lr, epoch_lim and componente

df_summary = df_results.group_by(["capa", "neurona","umbral", "lr", "epoch_lim", "componente", "func"]).agg(pl.col("error_rate").mean())
df_summary


In [ ]:
df_summary.write_csv('/content/buckets/b1/DMA/NuestrasCaras/corridasResumen100x100.csv')

In [ ]:
## Test reciclado de epochs 2 capas

#inicio2 = datetime.datetime.now()
seeds = [200177, 1999, 1996, 1949] # seeds a utilizar
learning_rates = [0.2] #learning rates a probar
epochs = [100, 300, 500, 700, 1000, 1500, 2000, 2300, 2500] #Límites de epochs a usar, PONER EN ORDEN CRECIENTE
umbrales = [0.0001] # umbrales de mse a probar
capa1 = [13, 15, 20, 25, 30] # capas a usar, sin contar la última oculta, o sea, la red va a tener capa[i] + 1 (verificamos que usar una más overfitea mucho)
capa2 = [13, 15, 20, 25, 30] # Neuronas a usar en las capas declaradas anteriormente, por ahora dejamos fija la última oculta con 13 (a revisar)
isomap_comp = [70, 80] # n_components de isomap a probar, por ahora mantenemos el número de vecinos fijos
contador = 0 #Contador solamente para guardar los diferentes pkl de las redes
check_iter_list = 'list_of_iters' in locals()
func_out = ['logsig']

# Creamos las listas de nombres y caras para usar en la red
faces = [sublista[0] for sublista in caras]
names = [sublista[1] for sublista in caras]

# Creación del df para guardar los resultados que vamos obteniendo (y no morir en el intento)
# seeds <- epochs <- lr <- umbral <- neuronas <- capas (ToDo: revisar para reciclar epochs y particiones de seed, ahora está haciendo uno por cada iter)

data = {'seed': [], 'capa1': [], 'capa2': [], 'umbral': [], 'lr': [], 'epoch': [], 'epoch_lim': [],  'componente': [], 'error_rate': [], 'func': []}
df_results = pl.DataFrame(data)

df_results = df_results.with_columns([
    pl.col("epoch").cast(pl.Int64),
    pl.col("epoch_lim").cast(pl.Int64),
    pl.col("seed").cast(pl.Int64),
    pl.col("capa1").cast(pl.Int64),
    pl.col("capa2").cast(pl.Int64),
    pl.col("componente").cast(pl.Int64),
    pl.col("error_rate").cast(pl.Float64),
    pl.col("umbral").cast(pl.Float64),
    pl.col("lr").cast(pl.Float64),
    pl.col("func").cast(pl.Utf8)
])


for n1 in capa1:
  for n2 in capa2:
    for umbral in umbrales:
      for lr in learning_rates:
        for mejor_n_components in isomap_comp:
          x_campos = campos_func(mejor_n_components)
          for seed in seeds:
            for f in func_out:
              contador += 1
              epoch_init = True
              skip = False
              for epoch_lim in epochs:
                # Train test
                random.seed(seed)
                train_caras, train_nombres, test_caras, test_nombres = separar_aleatorio(faces, names, 0.7)
                df_train, df_test = procesar_isomap(train_caras, train_nombres, test_caras, test_nombres, mejor_n_components, mejor_n_neighbors = 5)

                hidden_layers_sizes = [n1, n2]
                layers_func = ['logsig'] * (len(hidden_layers_sizes)) + [f]
                print(layers_func)

                # Entrenamiento

                # defino la red multiperceptron
                carpeta = "/content/buckets/b1/nn" + str(contador)  # cambiar con cada corrida
                semilla_red = seed  # define las rectas iniciales

                if epoch_init:
                  epoch_init = False
                  mp = multiperceptron()
                  mp.inicializar(
                      df=df_train, campos = x_campos, clase="y",  # especificaion del dataset
                      hidden_layers_sizes = hidden_layers_sizes,  # no va la capa final, solo hidden layers
                      layers_func=layers_func, # funciones de activacion de cada capa
                      semilla=semilla_red,
                      carpeta = carpeta
                      )

                  (epoch, MSE, train_error_rate) = mp.entrenar(
                    epoch_limit=epoch_lim,
                    MSE_umbral=umbral,
                    learning_rate=lr,
                    lr_momento=0.2,
                    save_frequency=100,
                    retomar=True)

                  if epoch < epoch_lim:  #Si llega al limite antes no tiene sentido seguir corriendo epochs
                    skip = True
                else: #reciclo el anterior
                  if not skip:
                    print('Hi')
                    new_lim = epoch_lim - epoch # el aumento en epochs
                    (epoch, MSE, train_error_rate) = mp.entrenar(
                      epoch_limit = epoch_lim, # aumento
                      MSE_umbral=umbral,
                      learning_rate=lr,
                      lr_momento=0.2,
                      save_frequency=100,
                      retomar=True)

                    if epoch < epoch_lim:
                      skip = True


                  else:
                    print('no')

                # Predicción
                (y_hat,y_raw, test_error_rate) = mp.predecir(df_new=df_test, campos=x_campos, clase='y')

                new_row = pl.DataFrame({'seed': [seed], 'capa1': [n1], 'capa2': [n2], 'umbral': [umbral], 'lr': [lr], 'epoch': [epoch], 'epoch_lim': [epoch_lim], 'componente': [mejor_n_components], 'error_rate': [test_error_rate], 'func':[f] })
                print(new_row)
                df_results = pl.concat([df_results, new_row])
                print(contador)


#fin2 = datetime.datetime.now()

In [ ]:
df_summary2 = df_results.group_by(["capa1", "capa2","umbral", "lr", "epoch_lim", "componente", "func"]).agg(pl.col("error_rate").mean())
df_summary2

In [ ]:
df_summary2.write_csv('/content/buckets/b1/DMA/NuestrasCaras/corridasResumen100x100Final.csv')

In [ ]:
df_results_in

In [ ]:
df_results.write_csv('/content/buckets/b1/DMA/NuestrasCaras/corridasult.csv')

In [ ]:
df_results

#Test caras externas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
fotos_prueba_path = '/content/drive/MyDrive/Fotos_Nuestras'  # Reemplaza con la ruta correcta

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

#Importar caras
def importar_fotos(path):
  '''
  Puedes acceder a cada foto y su nombre de carpeta así:
  for foto, nombre_carpeta in lista_fotos:
   print(f"Foto: {foto}, Carpeta: {nombre_carpeta}")
  '''
  lista_fotos = []
  for carpeta in os.listdir(fotos_prueba_path):
    carpeta_path = os.path.join(fotos_prueba_path, carpeta)
    if os.path.isdir(carpeta_path):
      for archivo in os.listdir(carpeta_path):
        archivo_path = os.path.join(carpeta_path, archivo)
        if os.path.isfile(archivo_path):
          try:
            img = cv2.imread(archivo_path, cv2.IMREAD_GRAYSCALE) # Importamos en gris para ahorrar memoria
            lista_fotos.append([img, carpeta])
            del img #para ir ganando puchitos de memoria, en caso de que sean más se puede unir al paso de detección
          except Exception as e:
            print(f"Error al abrir la imagen {archivo_path}: {e}")

  print(f"Se encontraron {len(lista_fotos)} fotos.")
  return lista_fotos




In [ ]:
imagenes = importar_fotos(fotos_prueba_path)

In [ ]:
# Descarga de Archivos
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt -O deploy.prototxt
!wget -q https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel -O res10_300x300_ssd_iter_140000.caffemodel

In [ ]:
# DNN Face Detector
face_net = cv2.dnn.readNetFromCaffe(
    'deploy.prototxt',
    'res10_300x300_ssd_iter_140000.caffemodel'
)


# Resultados
caras_test = []
errores_test = []

# ✅ Paso 3: Iterar sobre las imágenes
# imagenes = [...]  # <- asegurate que sea una lista de [imagen_array, etiqueta]
flag = 0

for imagen in imagenes:
    flag += 1
    print(f"Procesando imagen {flag}")

    img = imagen[0].copy()
    imagen[0] = np.array([]) #Para ir ahorrando memoria
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    h, w = img.shape[:2]

    # Paso 1: Detectar cara
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104, 177, 123))
    face_net.setInput(blob)
    detections = face_net.forward()

    found = False
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            found = True
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype("int")
            x1, y1, x2, y2 = max(x1, 0), max(y1, 0), min(x2, w), min(y2, h)

            cara = img[y1:y2, x1:x2]
            if cara.size == 0:
                continue

            # Paso 3: Guardar rostro recortado (usamos FaceMesh si querés más info)
            rostro = cv2.resize(cara, (50, 50))
            caras_test.append([rostro, imagen[1]])
            break

    if not found:
        errores_test.append([img, imagen[1]])

In [ ]:
for cara in caras_test:
  cv2_imshow(cara[0])

In [ ]:
for i in range(len(caras_test)):
  caras_test[i][0] = cv2.cvtColor(caras_test[i][0], cv2.COLOR_BGR2GRAY).reshape(1, 2500)

In [ ]:
#isomap
mejor_n_components = 80
# Creamos las listas de nombres y caras para usar en la red
train_caras = [sublista[0] for sublista in caras]
train_nombres = [sublista[1] for sublista in caras]
test_caras = [sublista[0] for sublista in caras_test]
test_nombres = [sublista[1] for sublista in caras_test]

df_train, df_test = procesar_isomap(train_caras, train_nombres, test_caras, test_nombres, mejor_n_components, mejor_n_neighbors = 10)

In [ ]:
# Test para caras externas


## Test reciclado de epochs 2 capas

#inicio2 = datetime.datetime.now()
seeds = [200177, 19999] # seeds a utilizar
learning_rates = [0.2, 0.05] #learning rates a probar
epochs = [100, 300, 500, 700, 1000, 1500, 2000] #Límites de epochs a usar, PONER EN ORDEN CRECIENTE
umbrales = [0.0001] # umbrales de mse a probar
capa1 = [20, 25, 30, 35, 40] # capas a usar, sin contar la última oculta, o sea, la red va a tener capa[i] + 1 (verificamos que usar una más overfitea mucho)
capa2 = [20, 25, 30, 35, 40]  # Neuronas a usar en las capas declaradas anteriormente, por ahora dejamos fija la última oculta con 13 (a revisar)
isomap_comp = [80] # n_components de isomap a probar, por ahora mantenemos el número de vecinos fijos
contador = 0 #Contador solamente para guardar los diferentes pkl de las redes
check_iter_list = 'list_of_iters' in locals()
func_out = ['logsig']

# Creamos las listas de nombres y caras para usar en la red
faces = [sublista[0] for sublista in caras]
names = [sublista[1] for sublista in caras]

# ToDo armar df_train

# Creación del df para guardar los resultados que vamos obteniendo (y no morir en el intento)
# seeds <- epochs <- lr <- umbral <- neuronas <- capas (ToDo: revisar para reciclar epochs y particiones de seed, ahora está haciendo uno por cada iter)

data = {'seed': [], 'capa1': [], 'capa2': [], 'umbral': [], 'lr': [], 'epoch': [], 'epoch_lim': [],  'componente': [], 'MSE': [], 'error_rate': [], 'func': []}
df_results = pl.DataFrame(data)

df_results = df_results.with_columns([
    pl.col("epoch").cast(pl.Int64),
    pl.col("epoch_lim").cast(pl.Int64),
    pl.col("seed").cast(pl.Int64),
    pl.col("capa1").cast(pl.Int64),
    pl.col("capa2").cast(pl.Int64),
    pl.col("componente").cast(pl.Int64),
    pl.col("error_rate").cast(pl.Float64),
    pl.col("MSE").cast(pl.Float64),
    pl.col("umbral").cast(pl.Float64),
    pl.col("lr").cast(pl.Float64),
    pl.col("func").cast(pl.Utf8)
])


for n1 in capa1:
  for n2 in capa2:
    for umbral in umbrales:
      for lr in learning_rates:
        for mejor_n_components in isomap_comp:
          x_campos = campos_func(mejor_n_components)
          for seed in seeds:
            for f in func_out:
              contador += 1
              epoch_init = True
              skip = False
              for epoch_lim in epochs:
                # Train test

                hidden_layers_sizes = [n1, n2]
                layers_func = ['logsig'] * (len(hidden_layers_sizes)) + [f]
                print(layers_func)

                # Entrenamiento

                # defino la red multiperceptron
                carpeta = "/content/buckets/b1/nn" + str(contador)  # cambiar con cada corrida
                semilla_red = seed  # define las rectas iniciales

                if epoch_init:
                  epoch_init = False
                  mp = multiperceptron()
                  mp.inicializar(
                      df=df_train, campos = x_campos, clase="y",  # especificaion del dataset
                      hidden_layers_sizes = hidden_layers_sizes,  # no va la capa final, solo hidden layers
                      layers_func=layers_func, # funciones de activacion de cada capa
                      semilla=semilla_red,
                      carpeta = carpeta
                      )

                  (epoch, MSE, train_error_rate) = mp.entrenar(
                    epoch_limit=epoch_lim,
                    MSE_umbral=umbral,
                    learning_rate=lr,
                    lr_momento=0.2,
                    save_frequency=100,
                    retomar=True)

                  if epoch < epoch_lim:  #Si llega al limite antes no tiene sentido seguir corriendo epochs
                    skip = True
                else: #reciclo el anterior
                  if not skip:
                    print('Hi')
                    new_lim = epoch_lim - epoch # el aumento en epochs
                    (epoch, MSE, train_error_rate) = mp.entrenar(
                      epoch_limit = epoch_lim, # aumento
                      MSE_umbral=umbral,
                      learning_rate=lr,
                      lr_momento=0.2,
                      save_frequency=100,
                      retomar=True)

                    if epoch < epoch_lim:
                      skip = True


                  else:
                    print('no')

                # Predicción
                (y_hat,y_raw, test_error_rate) = mp.predecir(df_new=df_test, campos=x_campos, clase='y')

                new_row = pl.DataFrame({'seed': [seed], 'capa1': [n1], 'capa2': [n2], 'umbral': [umbral], 'lr': [lr], 'epoch': [epoch], 'epoch_lim': [epoch_lim], 'componente': [mejor_n_components],'MSE':[MSE], 'error_rate': [test_error_rate],  'func':[f] })
                print(new_row)
                df_results = pl.concat([df_results, new_row])
                print(contador)




todo: pruebas con pocas neuronas por capas pero muchos epochs

In [ ]:
df_results.write_csv('/content/buckets/b1/DMA/NuestrasCaras/corridas_capas_50_fotosnati.csv')

In [ ]:
df_results

In [ ]:
df_results.group_by(["capa1", "capa2","umbral", "lr", "epoch_lim", "componente", "func"]).agg(pl.col("error_rate").mean())

In [ ]:
# prompt: generate a code to summarize the df_result df. It has to do a mean of the error_rate grouping by capa1, capa2, umbral, lr, epoch_lim and componente

df_summary = df_results.group_by(["capa", "neurona", , "umbral", "lr", "epoch_lim", "componente"]).agg(pl.col("error_rate").mean())
df_summary


#fotosnati

capa1	capa2	umbral	lr	epoch_lim	componente	error_rate

30	20	0.0001	0.2	1000	50	0.18965008992779794   #logsig

fin del test

In [ ]:
# prompt: necesito un codigo que me abra un csv desde drive. El path está en mi drive pero que pueda abrirlo otra persona tambien. el enlace es: https://drive.google.com/file/d/18pouLFeG9hFI3N_DYKV9NiyCcfhbkGux/view?usp=drive_link . tiene que usar polars. It can be in another's person drive

import polars as pl
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Replace with the actual path to your CSV file in your Google Drive
file_path = '/content/drive/MyDrive/your_folder/your_file.csv'  # UPDATE THIS PATH

try:
  df = pl.read_csv(file_path)
  print(df)
except Exception as e:
  print(f"Error reading the CSV file: {e}")


Todo:


*   Guardar MSE
*   Cambiar for del seed



### 2.2 Entrenamiento de la neural network = backpropagation

Aqui se hace el trabajo pesado de entrenar la red neuronal

Es necesario experimentar con


*   learning_rate
*   lr_momento
*   epoch_limit  y MSE_umbral



In [ ]:
# entreno la neural netowork con BackPropagation

# el entrenamiento
(epoch, MSE, train_error_rate) = mp.entrenar(
    epoch_limit=20000,
    MSE_umbral=0.0005,
    learning_rate=0.2,
    lr_momento=0.2,
    save_frequency=100,
    retomar=True)


#### Visualizacion de los resultados de la salida del entrenamiento de la red

In [ ]:
# las metricas basica de la red
print("epoch :", epoch)
print("MSE :", MSE)
print("train_error_rate :", train_error_rate)

In [ ]:
# la primera hidden layer
print("W :", mp.red["layer"][0]["W"])
print()
print("w0 :", mp.red["layer"][0]["w0"])

### 2.3 Entrenamiento en caso de retomar



*   Si se cortó el colab
*   Si quiero extender la corrida a mas epochs
*   Si quiero cambiar el learninh_rate
*   Si quiero cambiar el MSE_umbral



In [ ]:
(epoch, MSE, train_error_rate) = mp.entrenar(
    epoch_limit=1800, # aumento
    MSE_umbral=0.001,
    learning_rate=0.05,
    lr_momento=0.05,
    save_frequency=100,
    retomar=True)

Visualizacion de los resultados de salida de un post entrenamiento

In [ ]:
print("epoch :", epoch)
print("MSE :", MSE)
print("train_error_rate :", train_error_rate)

In [ ]:
# la primera hidden layer
print("W :", mp.red["layer"][0]["W"])
print()
print("w0 :", mp.red["layer"][0]["w0"])

## 3  Prediccion en los datos de Testing


Se muestran los datos de testing, que son distintos a los de training

### 3.1 Prediccion en caliente

In [ ]:
# aplico la red entrenada al dataset de testing

(y_hat,y_raw, test_error_rate) = mp.predecir(df_new=df_test, campos=campos, clase='y')

#### Visualizacion del error en testing

In [ ]:
print("error_rate (train, test): ",  train_error_rate, test_error_rate)

#### Visualizacion de la prediccion en testing

In [ ]:
tb_salida_test = pl.DataFrame( {"clase":df_test["y"], "pred":y_hat, "y_raw":y_raw })
tb_salida_test

In [ ]:
for i in range(len(tb_salida_test)):
    if not str(tb_salida_test[i]["clase"]) == str(tb_salida_test[i]["pred"]):
      print(tb_salida_test[i]["clase"], tb_salida_test[i]["pred"])

#### Cálculo de errores por persona

In [ ]:
# Paso 1: crear una columna booleana que indique si acertó o no
df = tb_salida_test.with_columns([
    (pl.col("clase") != pl.col("pred")).alias("error")
])

# Paso 2: agrupar por clase real y calcular tasa de error por clase
errores_por_clase = (
    df.group_by("clase")
    .agg([
        pl.len().alias("total"),
        pl.col("error").sum().alias("errores")
    ])
    .with_columns([
        (pl.col("errores") / pl.col("total")).alias("error_rate")
    ])
    .sort("error_rate", descending=True)
)
pl.Config.set_tbl_rows(20)
print(errores_por_clase)

In [ ]:
def analisis_fotos(fotos_reales, prediccion, fotos_error = True):
  for i in range(len(prediccion)):
    if ((fotos_error and prediccion['error'][i]) or (not(fotos_error) and not(prediccion['error'][i]))):
      print(f"La foto era de {prediccion['clase'][i]} y la red dijo que era de {prediccion['pred'][i]}")
      plt.figure(figsize=(1, 1))
      plt.imshow(test_caras[i].reshape(30, 30), cmap='gray')
      plt.axis('off')
      plt.show()

In [ ]:
analisis_fotos(data_testing, df, True)


## 4 Prediccion en datos NUEVOS


*   La red fue entrenada en el pasado, y se grabó al drive
*   Ya no esta disponible la sesion donde se entreno
*   No quiero volver a entrenar de cero

In [ ]:
# cargo datos NUEVOS
df_new = pl.read_csv('https://storage.googleapis.com/open-courses/austral2025-af91/nuevos02.txt', separator='\t')
df_new.shape

In [ ]:
# cargo modelo grabado y lo aplico a los datos nuevos

carpeta = "/content/buckets/b1/nn01/"  # la carpeta del modelo entrenado

mp_frio = multiperceptron()
(epoch, MSE, train_error_rate) = mp_frio.cargar_modelo(carpeta)

(y_hat, y_raw, new_error_rate) = mp_frio.predecir(df_new=df_new, campos=['x1', 'x2'], clase='y')

#### Visualizacion del error modeloa aplicado a datos nuevos

In [ ]:
print("error_rate (train, new): ",  train_error_rate, new_error_rate)

#### Visualizacion de la prediccion en datos nuevos

In [ ]:
tb_salida_new = pl.DataFrame( {"clase":df_new["y"], "pred":y_hat, "y_raw":y_raw })
tb_salida_new